# word2vec 속도 개선

## Embedding 계층

In [2]:
import numpy as np

In [70]:
W = np.arange(21).reshape(7,3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [7]:
W[5]

array([15, 16, 17])

In [9]:
idx = np.array([1,0,3,0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [10]:
class Embedding:
    def __init__(self, W) :
        self.params = [W] # 리스트 안에 담기
        self.grads = [np.zeros_like(W)]
    
    def forward(self, idx) :
        W, = self.params # 리스트  에서 해제
        self.idx = idx
        out = W[idx]
        return out
    
#     def backward(self, dout):
#         dW, = self.grads
#         dW[...] = 0
#         dw[self.idx] = dout
#         return None
# idx에 따라 중복할당이 될 수도 있기에 좋지 않은 방법

def backward(self, dout):
    dW = self.grads
    dW[...]=0 # dW 모양유지하면서 원소들 0으로 대체
    
    for i, word_id in enumerate(self.idx):
        dw[word_id] +=dout[i]
        # 혹은
        # np.add.at(dW, self.idx, dout)
        return None

## 다중분류에서 이진분류로, EmbeddingDot 구현

In [36]:
class EmbeddingDot :
    def __init__(self,W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
    
    def forward(self, h, idx) :
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis =1)
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0],1)
        dtarget_W = dout*h
        self.embed.backward(dtarget_W)
        dh = dout*target_W
        return dh

## 네거티브 샘플링

### 네거티브 샘플링 기법

In [39]:
np.random.choice(10)

2

In [40]:
np.random.choice(10)

9

In [53]:
words = ['you','say','goodbye','I','hello','.']
np.random.choice(words) # 실행할 때마다 바뀜

'hello'

In [48]:
# 5개만 무작위로 샘플링(중복있음)
np.random.choice(words, size=5)

array(['you', 'hello', 'you', 'I', 'say'], dtype='<U7')

In [51]:
np.random.choice(words, size=5, replace = False) # 중복없음

array(['I', 'say', 'hello', 'you', 'goodbye'], dtype='<U7')

In [61]:
# 확률 분포에 따라 샘플링
p = [0.5,0.1,0.05,0.2,0.05,0.1]
np.random.choice(words,p=p)

'you'

In [64]:
# 확률분포에 0.75 제곱을 하는 것을 권장 -> 확률이 낮은 단어의 확률을 살짝 높여 버리지 않기 위해
p = [0.7,0.29,0.01]
new_p = np.power(p,0.75)
new_p /= np.sum(new_p)
print(new_p)

[0.64196878 0.33150408 0.02652714]


In [74]:
sample_size = 5
embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size+1)]

### 네거티브 샘플링 구현

In [85]:
import sys
sys.path.append('..')
from common import negative_sampling_layer
from common.negative_sampling_layer import UnigramSampler

In [90]:
corpus = np.array([0,1,2,3,4,1,2,3])
power = 0.75
sample_size = 2
sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1,3,0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)
# 첫번째, 1이 아닌 애들, 두번째 3이 아닌 애들, 세번째 0이 아닌 애들 2개씩

[[0 4]
 [0 1]
 [4 3]]


In [69]:
class NegativeSamplingLoss :
    def __init__(self, W, corpus, power = 0.75, sample_size = 5) : # 출력측 가중치 W, 부정적 예의 샘플링  횟수 sample_size
        self.samp_size = sample_size
        self.sampler=UnigramSampler(corpus, power, sample_size) # UnigramSampler 클래스 인스턴스 변수에 지정
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size+1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size+1)] 
        # sample size(부정) + target[긍정] 더한 개수 만큼 embed_dot_layers, loss_layer 층 생성
        self.params, self.grads = [], []
        self.params += layer.params
        self.grads += layer.grads
    
    def forward(self, h, target) :
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target) 
        # UnigramSampler 클래스 내의 get~ 메서드 사용, target 외에 단어로 생플링 
        
        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h,target)
        # 첫번째 층에서 target과 h 값 계산
        correct_label = np.ones(batch_size, dtype = np.int32)
        # batch size 만큼 1로 구성된 배열 생성
        loss = self.loss_layers[0].forward(score, correct_label)
        # 1값과 score 값 간의 차이(손실)계산
        
        # 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32)
        # 오답은 0으로 구성된 배열 구성
        for i in range(self,sample_size) :
            negative_target = negative_sample[:,i]
            score = self.embed_dot_layers[1+i].forward(h,negative_target)
            loss += self.loss_layers[1+i].forward(score, negative_label)
        
        return loss
    
    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)
            
        return dh

## 개선판 word2vec 학습

### CBOW 모델 구현

In [116]:
from common.layers import Embedding
from common.negative_sampling_layer import NegativeSamplingLoss

class CBOW :
    def __init__(self, vocab_size, hidden_size, window_size, corpus) :
        V,H = vocab_size, hidden_size
        
        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V,H).astype('f')
        W_out = 0.01 * np.random.randn(V,H).astype('f') # 임베딩 계층을 거치기에 그대로 형태 유지
        
        # 계층 생성
        self.in_layers = []
        for i in range(2*window_size) : # 임베딩 계층을 2*windowsize 작성하여 in_layer에 보관
            layer = Embedding(W_in) # 임베딩 계층 사용
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5) # 네거티브 샘플링 및 손실 계층 생성
        
        # 모든 가중치와 기울기를 배열에 모은다
        layers = self.in_layers + [self.ns_loss] # 임베딩계층 후에 네거티브 샘플링 계층 붙이기
        self.params, self.grads = [], []
        for layer in layers :
            self.params +=layer.params
            self.grads += layer.grads
            
        # 인스턴스 변수에 단어의 분산표현을 저장한다
        self.word_vecs = W_in
        
    def forward(self, contexts, target) :
        h =0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:,i]) # 맥락 위치 인덱싱
        h *= 1/ len(self.in_layers)
        loss = self.ns_loss.forward(h,target)
        return loss
    
    def backward(self, dout=1) :
        dout = self.ns_loss.backward(dout)
        dout *= 1/len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

### CBOW 모델학습코드

In [106]:
sys.path.append('..')
import pickle
from common.trainer import Trainer
from common. optimizer import Adam
from common.util import create_contexts_target, to_cpu, to_gpu
import ptb

In [109]:
# 하이퍼 파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)

In [111]:
word_to_id

{'aer': 0,
 'banknote': 1,
 'berlitz': 2,
 'calloway': 3,
 'centrust': 4,
 'cluett': 5,
 'fromstein': 6,
 'gitano': 7,
 'guterman': 8,
 'hydro-quebec': 9,
 'ipo': 10,
 'kia': 11,
 'memotec': 12,
 'mlx': 13,
 'nahb': 14,
 'punts': 15,
 'rake': 16,
 'regatta': 17,
 'rubens': 18,
 'sim': 19,
 'snack-food': 20,
 'ssangyong': 21,
 'swapo': 22,
 'wachter': 23,
 '<eos>': 24,
 'pierre': 25,
 '<unk>': 26,
 'N': 27,
 'years': 28,
 'old': 29,
 'will': 30,
 'join': 31,
 'the': 32,
 'board': 33,
 'as': 34,
 'a': 35,
 'nonexecutive': 36,
 'director': 37,
 'nov.': 38,
 'mr.': 39,
 'is': 40,
 'chairman': 41,
 'of': 42,
 'n.v.': 43,
 'dutch': 44,
 'publishing': 45,
 'group': 46,
 'rudolph': 47,
 'and': 48,
 'former': 49,
 'consolidated': 50,
 'gold': 51,
 'fields': 52,
 'plc': 53,
 'was': 54,
 'named': 55,
 'this': 56,
 'british': 57,
 'industrial': 58,
 'conglomerate': 59,
 'form': 60,
 'asbestos': 61,
 'once': 62,
 'used': 63,
 'to': 64,
 'make': 65,
 'kent': 66,
 'cigarette': 67,
 'filters': 68,
 'h

In [112]:
corpus

array([ 0,  1,  2, ..., 39, 26, 24])

In [114]:
contexts[0] # 윈도우 사이즈가 5라 id = 5 부터 시작

array([ 0,  1,  2,  3,  4,  6,  7,  8,  9, 10])

In [124]:
class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embedding 계층 사용
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss


    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None


In [125]:
# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

| 에폭 1 |  반복 1 / 9295 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 21 / 9295 | 시간 1[s] | 손실 4.16
| 에폭 1 |  반복 41 / 9295 | 시간 3[s] | 손실 4.15
| 에폭 1 |  반복 61 / 9295 | 시간 5[s] | 손실 4.12
| 에폭 1 |  반복 81 / 9295 | 시간 6[s] | 손실 4.05
| 에폭 1 |  반복 101 / 9295 | 시간 8[s] | 손실 3.92
| 에폭 1 |  반복 121 / 9295 | 시간 9[s] | 손실 3.78
| 에폭 1 |  반복 141 / 9295 | 시간 11[s] | 손실 3.63
| 에폭 1 |  반복 161 / 9295 | 시간 13[s] | 손실 3.49
| 에폭 1 |  반복 181 / 9295 | 시간 15[s] | 손실 3.37
| 에폭 1 |  반복 201 / 9295 | 시간 17[s] | 손실 3.26
| 에폭 1 |  반복 221 / 9295 | 시간 18[s] | 손실 3.17
| 에폭 1 |  반복 241 / 9295 | 시간 21[s] | 손실 3.08
| 에폭 1 |  반복 261 / 9295 | 시간 23[s] | 손실 3.03
| 에폭 1 |  반복 281 / 9295 | 시간 24[s] | 손실 2.99
| 에폭 1 |  반복 301 / 9295 | 시간 26[s] | 손실 2.93
| 에폭 1 |  반복 321 / 9295 | 시간 28[s] | 손실 2.88
| 에폭 1 |  반복 341 / 9295 | 시간 29[s] | 손실 2.83
| 에폭 1 |  반복 361 / 9295 | 시간 31[s] | 손실 2.81
| 에폭 1 |  반복 381 / 9295 | 시간 32[s] | 손실 2.78
| 에폭 1 |  반복 401 / 9295 | 시간 34[s] | 손실 2.76
| 에폭 1 |  반복 421 / 9295 | 시간 36[s] | 손실 2.75
| 에폭 1 |  반복 441 / 9295

| 에폭 1 |  반복 3541 / 9295 | 시간 308[s] | 손실 2.40
| 에폭 1 |  반복 3561 / 9295 | 시간 309[s] | 손실 2.41
| 에폭 1 |  반복 3581 / 9295 | 시간 311[s] | 손실 2.41
| 에폭 1 |  반복 3601 / 9295 | 시간 313[s] | 손실 2.42
| 에폭 1 |  반복 3621 / 9295 | 시간 314[s] | 손실 2.39
| 에폭 1 |  반복 3641 / 9295 | 시간 316[s] | 손실 2.42
| 에폭 1 |  반복 3661 / 9295 | 시간 318[s] | 손실 2.43
| 에폭 1 |  반복 3681 / 9295 | 시간 319[s] | 손실 2.40
| 에폭 1 |  반복 3701 / 9295 | 시간 321[s] | 손실 2.42
| 에폭 1 |  반복 3721 / 9295 | 시간 322[s] | 손실 2.43
| 에폭 1 |  반복 3741 / 9295 | 시간 324[s] | 손실 2.40
| 에폭 1 |  반복 3761 / 9295 | 시간 326[s] | 손실 2.39
| 에폭 1 |  반복 3781 / 9295 | 시간 327[s] | 손실 2.39
| 에폭 1 |  반복 3801 / 9295 | 시간 329[s] | 손실 2.42
| 에폭 1 |  반복 3821 / 9295 | 시간 331[s] | 손실 2.43
| 에폭 1 |  반복 3841 / 9295 | 시간 332[s] | 손실 2.42
| 에폭 1 |  반복 3861 / 9295 | 시간 334[s] | 손실 2.42
| 에폭 1 |  반복 3881 / 9295 | 시간 335[s] | 손실 2.40
| 에폭 1 |  반복 3901 / 9295 | 시간 337[s] | 손실 2.42
| 에폭 1 |  반복 3921 / 9295 | 시간 339[s] | 손실 2.40
| 에폭 1 |  반복 3941 / 9295 | 시간 340[s] | 손실 2.44
| 에폭 1 |  반복 

KeyboardInterrupt: 

In [ ]:
# 나중에 사용할 수 있도록 데이터 저장
word_vecs = model.word_vecs
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'
wirh open(pkl_file, 'wb') as f :
    pickle.dump(params, f, -1)

### CBOW 모델 평가

In [127]:
from common.util import most_similar

pkl_file = 'cbow_params.pkl'

with open(pkl_file, 'rb') as f :
    params = pickle.load(f)

In [130]:
word_vecs = params['word_vecs']
word_to_id = params['word_to_id']
id_to_word = params['id_to_word']

querys = ['you', 'year', 'car', 'toyota']
for query in querys :
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)


[query] you
 we: 0.6103515625
 someone: 0.59130859375
 i: 0.55419921875
 something: 0.48974609375
 anyone: 0.47314453125

[query] year
 month: 0.71875
 week: 0.65234375
 spring: 0.62744140625
 summer: 0.6259765625
 decade: 0.603515625

[query] car
 luxury: 0.497314453125
 arabia: 0.47802734375
 auto: 0.47119140625
 disk-drive: 0.450927734375
 travel: 0.4091796875

[query] toyota
 ford: 0.55078125
 instrumentation: 0.509765625
 mazda: 0.49365234375
 bethlehem: 0.47509765625
 nissan: 0.474853515625
